# Lightning Network Liquidity Auction

In [ ]:
%load_ext autoreload
%autoreload 2

## Partially Signed Bitcoin Transaction Example (PSBT)

House will facilitate a multi-sig transaction between Alice and Bob that has Alice send her funds to Bob.

**If you need to increase your wallet balance**

Run `rpc.getnewaddress()` then set `"--miningaddr=..."` in `btcd` docker-compose `command: ...`. Then generate some blocks to stack sats: `next(rpc.generate(400))`

### Create & Fund a Multisig

In [ ]:
FUND_WALLETS = False

In [ ]:
import os
import binascii
import bitcoin
import bitcoin.wallet as bw
import bitcoin.core as bc
import bitcoin.core.script as bs
from rpc import Proxy, Config

bitcoin.SelectParams('regtest')

# API: https://developer.bitcoin.org/reference/rpc/
config = Config(
    rpcuser=os.environ['BTCD_RPCUSER'],
    rpcpassword=os.environ['BTCD_RPCPASS'],
    rpcconnect='bitcoind',
    rpcport=18443
)
rpc_house = Proxy(config._replace(rpcconnect='bitcoind-house'))
rpc_alice = Proxy(config._replace(rpcconnect='bitcoind-alice'))
rpc_bob = Proxy(config._replace(rpcconnect='bitcoind-bob'))

def print_balances():
    print(f"Wallet Balance (House): {rpc_house.getbalance() / bc.COIN:.2f} BTC")
    print(f"Wallet Balance (Alice): {rpc_alice.getbalance() / bc.COIN:.2f} BTC")
    print(f"Wallet Balance (Bob): {rpc_bob.getbalance() / bc.COIN:.2f} BTC")

In [ ]:
junk_address = bw.CBitcoinAddress("2MyJKxYR2zNZZsZ39SgkCXWCfQtXKhnWSWq")

house = rpc_house.getnewaddress()
alice = rpc_alice.getnewaddress()
bob = rpc_bob.getnewaddress()

if FUND_WALLETS:
    _ = rpc_house.generatetoaddress(400, junk_address)
    _ = rpc_house.generatetoaddress(10, alice)

# Coinbase maturity period requires 100 confirmations
# (https://bitcoin.stackexchange.com/a/100980/59383)
# _ = rpc_house.generatetoaddress(200, junk_address)
print_balances()

Create multi-sig and send BTC to it.

In [ ]:
pubHouse = rpc_house.getaddressinfo(house)["pubkey"]
pubAlice = rpc_alice.getaddressinfo(alice)["pubkey"]
pubBob = rpc_bob.getaddressinfo(bob)["pubkey"]

# XXX: It seems as though the creator of the PSBT needs to be one of
#      the signers; that is why the house is here as part of a 3x3.
multi = rpc_house.addmultisigaddress(3, [pubHouse, pubAlice, pubBob], "", "bech32")

# Make wallets treat payments to multisig as contributing to the
# watch-only balance.
rpc_alice.importaddress(multi["address"])
rpc_bob.importaddress(multi["address"])

# Give money to the multisig output address
_ = rpc_alice.sendtoaddress(multi["address"], 2 * bc.COIN, subtractfeefromamount=True)
_ = rpc_house.generatetoaddress(100, junk_address)
print_balances()

### Spend Multisig via a PSBT

Create PSBT to send BTC from multi-sig, have all parties sign it.

In [ ]:
options = {
    "feeRate": 10_000 / bc.COIN,
    "subtractFeeFromOutputs": [0],
    "includeWatching": True
}

psbt = rpc_house._call(
    "walletcreatefundedpsbt",
    [],
    [{str(bob): 2}],  # 2 BTC
    0,
    options,
    None
)

# TODO: Before signing, Alice and Bob should inspect the psbt (by decoding)
#       to see if it has the appropriate input/output/amount/fee.
psbt_house = rpc_house._call("walletprocesspsbt", psbt["psbt"], True, "ALL", None)
psbt_alice = rpc_alice._call("walletprocesspsbt", psbt["psbt"], True, "ALL", None)
psbt_bob = rpc_bob._call("walletprocesspsbt", psbt["psbt"], True, "ALL", None)

psbt = rpc_house.combinepsbt(
    [psbt_house["psbt"], psbt_alice["psbt"], psbt_bob["psbt"]]
)
psbt = rpc_house._call("finalizepsbt", psbt, True)

assert psbt["complete"]

Publish spend from multisig transaction

In [ ]:
unhexlify = lambda h: binascii.unhexlify(h.encode("utf8"))
fund_tx = bc.CTransaction.deserialize(unhexlify(psbt["hex"]))

_ = rpc_house.sendrawtransaction(fund_tx)
rpc_house.generatetoaddress(6, junk_address)
print_balances()

## Use House to Coordinate LND Channel Creation

In [ ]:
# ...

## Use CLOB to Coordinate Orders

In [ ]:
from order_book import (
    Conversions,
    OrderEngine,
    Order,
    OrderSide as Side,
    OrderAccount as Account,
)

order_engine = OrderEngine()

alice = Account(
    # lncli --network=simnet newaddress np2wkh
    pubkey="rYjScWYTmZQqbhR7v2AKw6JY8QfYFkxe11",
    # lncli --network=simnet getinfo | grep identity_pubkey
    node_endpoint="03a1293196bc7b42669514a65956030d00a8422a0b962d36dc89c91e0a94561af3@localhost:19735"
)
bob = Account(
    pubkey="rbx1izzXVfY9WY6zBnD2Cg7MqfVzD3rQts",
    node_endpoint="02e8ba8c0fbd0b924992466df843364efc28db0f3592e9bd180d7c31aada073084@localhost:29735"
)
buy_order = Order(
    side=Side.BID,
    quantity=Conversions.btc_to_order_quantity(0.001),
    per_unit_fees=80,
    by=bob
)
sell_order = Order(
    side=Side.ASK,
    quantity=Conversions.btc_to_order_quantity(0.001),
    per_unit_fees=80,
    by=alice
)

bid_order_id = order_engine.limit(buy_order)
print(f"bid made; (bid) {bid_order_id} filled={not order_engine.book.has(bid_order_id)}")

ask_order_id = order_engine.limit(sell_order)
print(f"ask made; (ask) {ask_order_id} filled={not order_engine.book.has(ask_order_id)}")

print(f"(bid) {bid_order_id} filled={not order_engine.book.has(bid_order_id)}")
print(f"(ask) {ask_order_id} filled={not order_engine.book.has(ask_order_id)}")